# Проект для «Викишоп» BERT

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [13]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import torch
import transformers
from tqdm import notebook
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [14]:
try:
    df = pd.read_csv('toxic_comments.csv')
    print('датасет загружен локально')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
    print('датасет загружен с сервера')

датасет загружен локально


In [15]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [30]:
# посмотрим на распределение по кол-ву слов
df['len'] = df['text'].str.count('')
df.describe()

/var/folders/n5/_dbcy3_n789_1_zt9g95px240000gn/T/ipykernel_57774/3784451093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len'] = df['text'].str.count('')


,Unnamed: 0,toxic,len
count,2000.000000,2000.00000,2000.000000
mean,999.500000,0.10500,400.337500
std,577.494589,0.30663,593.237975
min,0.000000,0.00000,20.000000
25%,499.750000,0.00000,96.000000
50%,999.500000,0.00000,210.000000
75%,1499.250000,0.00000,460.250000
max,1999.000000,1.00000,4909.000000


/var/folders/n5/_dbcy3_n789_1_zt9g95px240000gn/T/ipykernel_57774/1607224393.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['len'] = df['text'].str.count('')


,Unnamed: 0,toxic,len
count,2000.000000,2000.00000,2000.000000
mean,999.500000,0.10500,400.337500
std,577.494589,0.30663,593.237975
min,0.000000,0.00000,20.000000
25%,499.750000,0.00000,96.000000
50%,999.500000,0.00000,210.000000
75%,1499.250000,0.00000,460.250000
max,1999.000000,1.00000,4909.000000


Видим, что 75 квантиль это 460 слов в 1 сообщении, а значит можно ограничить Берту max_length до 512 - много не потеряем.

In [31]:
# Возьмем не весь датасет - так как локально мощности не хватит даже с М1 и подключенным GPU
df = df[:2000]
# df.reset_index(drop=True, inplace=True)

In [18]:
# Возьмем Бертовскую токсик модель - будем опираться на нее
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')

In [19]:
# Возьмем Бертовскую токсик модель
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# Проставим токены
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

In [21]:
# Уравниловка
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [22]:
# Размер масок
attention_mask.shape

(2000, 512)

In [23]:
# Собственно эмбединги
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/20 [00:00<?, ?it/s]

In [25]:
# выборки
features = np.concatenate(embeddings)
target = df['toxic']

In [26]:
# размеры выборок
target.shape, features.shape

((2000,), (2000, 768))

In [27]:
# сплит
features_train, features_test, target_train, target_test = train_test_split \
    (features, target, test_size = 0.5, random_state = 12345)

In [28]:
# Линейная логистическая модель и метрика
model_reg = LogisticRegression()
model_reg.fit(features_train, target_train)
predicts = model_reg.predict(features_train)
predicts_test = model_reg.predict(features_test)
print(f1_score(predicts, target_train))
print(f1_score(predicts_test, target_test))

0.9954751131221719
0.9253731343283583


/Users/pavelbekirov/opt/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Выводы
Подобрали модель Бертом и логистической регрессией которая дает результат 0.92 на тесте.
Мы использовали маленькую часть выборки для ускорения процесса, но использую хорошие мощности
можно перебрать и все объекты.

## Чек-лист проверки

    - [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны